In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import pandas as pd
import numpy as np
from lstm.dataset import SiburDataset
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import pickle

In [40]:
DATA_DIR = pathlib.Path(".")
DATA_FILE = "sc2021_train_deals.csv"
AGG_COLS = ["material_code", "company_code", "country", "region",
            "manager_code"]
RS = 82736

In [35]:
data = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), parse_dates=["month", "date"])

In [36]:
data.head()

,material_code,company_code,country,region,manager_code,month,material_lvl1_name,material_lvl2_name,material_lvl3_name,contract_type,date,volume
0,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-01,43.0
1,197,0,Китай,Китай,16350,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,95.0
2,794,2162,Казахстан,Атырауская обл.,10942,2018-01-01,Базовые полимеры,ПП,ПП,Контракт,2018-01-02,57.0
3,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,21.0
4,133,0,Китай,Китай,17745,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,150.0


In [41]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)

In [42]:
group_ts.shape

(941, 31)

In [43]:
agg_cols = ["material_code", "company_code", "country", "region",
                    "manager_code", "material_lvl1_name", "material_lvl2_name",
                    "material_lvl3_name", "contract_type"]
group_ts = data.groupby(agg_cols + ["month"])["volume"].sum().unstack(fill_value=0)
group_ts.shape

(943, 31)

In [44]:
row = group_ts.iloc[0]

In [45]:
for ix, val in zip(row.index, row):
    vector = list(row.name) + [ix.month]

In [46]:
vector

[124,
 7278,
 'Россия',
 'Респ. Татарстан',
 17460,
 'Базовые полимеры',
 'ПЭ',
 'ПЭНП',
 'Contract + Spot',
 7]

In [49]:
encoder = OneHotEncoder()
data['month_'] = data['date'].dt.month
agg_cols = ["material_code", "company_code", "country", "region",
            "manager_code", "material_lvl1_name", "material_lvl2_name",
            "material_lvl3_name", "contract_type", "month_"]
encoder.fit(data[agg_cols])

OneHotEncoder()

In [50]:
encoder.transform([vector]).toarray().flatten().shape

(538,)

In [119]:
with open('./lstm/ohe_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

dataset = SiburDataset(
    data=data,
    encoder=encoder,
    period={
        'start': '2018-01-01',
        'end': '2019-03-01'
    })

In [127]:
X, target = next(iter(dataset))

In [121]:
for X, target in tqdm(dataset):
    pass

100%|████████████████████████████████████████| 943/943 [00:06<00:00, 154.92it/s]


In [128]:
X.shape

torch.Size([8, 527])

In [129]:
target

tensor([200.])

In [23]:
with open('ohe_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)